<a href="https://colab.research.google.com/github/eisbetterthanpi/pytorch/blob/main/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title data
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

train_data = datasets.FashionMNIST(root="data", train=True, download=True,transform=ToTensor(),)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor(),)
# print(training_data)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)
# test_dataloader: #image, label


trainiter = iter(train_loader)
images, labels = next(trainiter)
print(images.shape) # [64, 1, 28, 28] (batch size, num channels, height, width)


100%|██████████| 26421880/26421880 [00:02<00:00, 12417348.64it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 211706.02it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3971103.84it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20603317.74it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

torch.Size([64, 1, 28, 28])


In [14]:
# @title model
import torch
from torch import nn

# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.lin = nn.Sequential(
            nn.Linear(input_dim, 512), #apply linear transformation
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes, bias=False),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.lin(x)
        return logits

device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNetwork(28*28, 10).to(device) # create an instance and move it to device (cache?)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (lin): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=False)
    (5): Softmax(dim=1)
  )
)


In [6]:
# check model implimentation
X = torch.rand(1, 28, 28, device=device)
pred = model(X)
y_pred = pred.argmax(1)
print(f"Predicted class: {y_pred.item()}")

Predicted class: 7


In [7]:
# @title train test function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X) # Compute prediction error
        loss = loss_fn(pred, y)

        loss.backward() # Backpropagation
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [15]:
# train
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth") # save model weights to 'model.pth'
# model = NeuralNetwork(28*28, 10) # create new model
# model.load_state_dict(torch.load("model.pth")) # load model weights from 'model.pth'


Epoch 1
-------------------------------
loss: 2.301691  [    0/60000]
loss: 2.301030  [ 6400/60000]
loss: 2.299149  [12800/60000]
loss: 2.299250  [19200/60000]
loss: 2.296257  [25600/60000]
loss: 2.293504  [32000/60000]
loss: 2.294422  [38400/60000]
loss: 2.289661  [44800/60000]
loss: 2.288761  [51200/60000]
loss: 2.284579  [57600/60000]
Test Error: 
 Accuracy: 38.5%, Avg loss: 2.282740 

Epoch 2
-------------------------------
loss: 2.283746  [    0/60000]
loss: 2.282482  [ 6400/60000]
loss: 2.271261  [12800/60000]
loss: 2.273700  [19200/60000]
loss: 2.252074  [25600/60000]
loss: 2.225666  [32000/60000]
loss: 2.235632  [38400/60000]
loss: 2.186033  [44800/60000]
loss: 2.209526  [51200/60000]
loss: 2.153451  [57600/60000]
Test Error: 
 Accuracy: 35.2%, Avg loss: 2.157811 

Epoch 3
-------------------------------
loss: 2.196000  [    0/60000]
loss: 2.167519  [ 6400/60000]
loss: 2.126631  [12800/60000]
loss: 2.125158  [19200/60000]
loss: 2.011139  [25600/60000]
loss: 2.048779  [32000/600

In [22]:
classes = ["T-shirt/top","Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot",]

model.eval()

import random
n=random.randint(0,1000)
print(n)
x, y = test_data[n][0], test_data[n][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')




876
Predicted: "Ankle boot", Actual: "Ankle boot"
